In [1]:
from transformers import AutoModelForTokenClassification

In [2]:
model_id = "meta-llama/Llama-3.2-3B"

model = AutoModelForTokenClassification.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model

LlamaForTokenClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    

In [4]:
from src.metrics.attention_geometry.lightweight import AttentionExtractor, SymmetryScore, DirectionalityScore

extractor = AttentionExtractor(
    model=model,
    q_path="model.layers[layer_idx].self_attn.q_proj.weight",
    k_path="model.layers[layer_idx].self_attn.k_proj.weight",
    attention_type="grouped"
)

score = SymmetryScore(extractor)

In [5]:
res = score()

In [8]:
import numpy as np

np.array(res).mean()

0.014972120523452759